In [ ]:
import boto3
import concurrent.futures
import time
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from botocore.exceptions import ClientError

# Initialize the Timestream client
client = boto3.client('timestream-query', region_name='us-east-2')

# List of different Timestream queries
queries = [
       f'select memory from "devops"."sample_devops" where time > ago(10m) and hostname=\'host1\' order by time desc limit 1',
     ]

# Function to run a single query and measure its duration
def run_query(query, worker_id):
    start_time = time.time()
    try:
        response = client.query(QueryString=query)
        duration = time.time() - start_time
        return worker_id, response, duration, None
    except ClientError as e:
        duration = time.time() - start_time
        if e.response['Error']['Code'] == 'ThrottlingException':
            return worker_id, None, duration, 'ThrottlingException'
        else:
            raise e

# Function to run queries in parallel
def run_parallel_queries(duration_seconds, queries, max_workers):
    end_time = time.time() + duration_seconds
    total_queries = 0
    query_durations = []
    throttling_count = 0

    print(f"\nStart time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        while time.time() < end_time:
            # Launch each query in parallel with a worker ID
            futures = {executor.submit(run_query, queries[i % len(queries)], i): i for i in range(max_workers)}

            # Process results
            for future in concurrent.futures.as_completed(futures):
                try:
                    worker_id, result, duration, error = future.result()
                    query_durations.append(duration)
                    total_queries += 1
                    if error == 'ThrottlingException':
                        throttling_count += 1
                    #print(f"Worker {worker_id}: Duration: {duration:.2f} seconds, results: {result}")
                except Exception as e:
                    print(f"Worker {worker_id}: Query failed: {e}")

    print(f"End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    if query_durations:
        p50 = np.percentile(query_durations, 50)
        p90 = np.percentile(query_durations, 90)
        p99 = np.percentile(query_durations, 99)
    else:
        p50 = p90 = p99 = None

    return total_queries, p50, p90, p99, throttling_count

# Number of threads (workers) to test
worker_counts = [7, 14, 21, 28, 42, 50, 60]
duration_seconds = 60

results = []

# Run the tests for different worker counts
for max_workers in worker_counts:
    total_queries, p50, p90, p99, throttling_count = run_parallel_queries(duration_seconds, queries, max_workers)
    results.append((max_workers, total_queries, p50, p90, p99, throttling_count))
    print(f"num_workers: {max_workers}")
    print(f"Total number of queries run in {duration_seconds} seconds: {total_queries}")
    print(f"p50 (50th percentile) of query durations: {p50:.2f} seconds")
    print(f"p90 (90th percentile) of query durations: {p90:.2f} seconds")
    print(f"p99 (99th percentile) of query durations: {p99:.2f} seconds")
    print(f"Throttling count: {throttling_count}")




In [ ]:
worker_counts = [result[0] for result in results]
total_queries = [result[1] for result in results]
p50s = [result[2] for result in results]
p90s = [result[3] for result in results]
p99s = [result[4] for result in results]
throttling_counts = [result[5] for result in results]

In [ ]:
print(worker_counts)

In [ ]:
print(total_queries)

In [ ]:
print(p50s)

In [ ]:
print(p90s)

In [ ]:
print(p99s)

In [ ]:
plt.figure(figsize=(12, 8))

# Plot latency percentiles
plt.subplot(3, 1, 1)
plt.plot(worker_counts, p50s, label='p50')
plt.plot(worker_counts, p90s, label='p90')
plt.plot(worker_counts, p99s, label='p99')
plt.xlabel('Number of Workers')
plt.ylabel('Latency (seconds)')
plt.title('Latency Percentiles')
plt.legend()

# Plot Queries Per Minute (QPM)
plt.subplot(3, 1, 2)
qpm = [q / (duration_seconds / 60) for q in total_queries]
plt.plot(worker_counts, qpm, label='Queries Per Minute (QPM)')
plt.xlabel('Number of Workers')
plt.ylabel('Queries Per Minute')
plt.title('Queries Per Minute')
plt.legend()

# Plot Throttling Counts
plt.subplot(3, 1, 3)
plt.plot(worker_counts, throttling_counts, label='Throttling Count', color='red')
plt.xlabel('Number of Workers')
plt.ylabel('Throttling Count')
plt.title('Throttling Count')
plt.legend()

plt.tight_layout()
plt.show()
